In [1]:
import os
import pickle
import random

In [2]:
root_dir = "python"
_, current_folder = os.path.split(os.getcwd())
if current_folder != root_dir:
    os.chdir("../")

In [3]:
path = os.environ.get("PATH")
additional_path = 'C:\\Users\\Michi\\Anaconda3\\envs\\python_v3-8;C:\\Users\\Michi\\Anaconda3\\envs\\python_v3-8\\Library\\mingw-w64\\bin;C:\\Users\\Michi\\Anaconda3\\envs\\python_v3-8\\Library\\usr\\bin;C:\\Users\\Michi\\Anaconda3\\envs\\python_v3-8\\Library\\bin;C:\\Users\\Michi\\Anaconda3\\envs\\python_v3-8\\Scripts;C:\\Users\\Michi\\Anaconda3\\envs\\python_v3-8\\bin;C:\\Users\\Michi\\Anaconda3\\condabin'
min_additional_path = "C:\\Users\\Michi\\Anaconda3\\envs\\python_v3-8\\Library\\bin;C:\\Users\\micha\\anaconda3\\envs\\ai38\\Library\\bin;"
combined_path = min_additional_path + path
os.environ["PATH"] = combined_path

In [4]:
import librosa
import sounddevice as sd


KeyboardInterrupt



In [ ]:
import torch
import numpy as np
from utils.augmentations_and_transforms import AddGaussianNoise, CyclicTemporalShift
import matplotlib.pyplot as plt

In [ ]:
silence = torch.zeros(50, 100)
plt.imshow(silence)

In [ ]:
noiseTransform = AddGaussianNoise(0., 1.)
noise = noiseTransform(silence)
plt.imshow(noise)
plt.colorbar()

In [ ]:
all_samples = np.array(torch.flatten(noise))
plt.hist(all_samples, 30)
all_samples.shape

# try out cyclic shift transform

In [ ]:
silence = torch.zeros(5,10)
click = silence
click[:, 3] = 1
click[2, 3] = 0
plt.imshow(click)

In [ ]:
shifter = CyclicTemporalShift()
shifted_click = shifter(click)
plt.imshow(shifted_click)

# test with MFCC data

In [ ]:
with open("data/Coswara_processed/pickles/participant_objects.pickle", "rb") as f:
            participants = pickle.load(f)
mfccs = participants[0].heavy_cough.MFCCs

In [ ]:
librosa.display.specshow(mfccs)
plt.xlabel("Time Frame")
plt.ylabel("MFCC")
plt.colorbar()

In [ ]:
heavy_noise_generator = AddGaussianNoise(0., 5.)
noisy_mfccs = heavy_noise_generator(torch.tensor(mfccs))
shifted_noisy_mfccs = shifter(noisy_mfccs)
librosa.display.specshow(np.array(shifted_noisy_mfccs))

# Audiomentations

In [ ]:
# from torch_audiomentations import Compose, AddBackgroundNoise, PitchShift, HighPassFilter, Gain, Shift
from audiomentations import Compose, AddGaussianNoise, PitchShift, HighPassFilter, Gain, Shift, TimeStretch, Trim

In [ ]:
participant_ids = os.listdir("data/Coswara_processed/Recordings/")
participant = random.choice(participant_ids)
audio_path = f"data/Coswara_processed/Recordings/{participant}/cough-heavy.wav"
audio_path

In [ ]:
audio, sr = librosa.load(audio_path)
audio = audio[1*sr:3*sr]

print(f"duration of recording: {round(len(audio)/sr, 1)}s")
sd.play(audio, sr)


In [ ]:
augmentations = Compose([
    AddGaussianNoise(min_amplitude=0.0001, max_amplitude=0.02, p=1.),
    PitchShift(min_semitones=-2, max_semitones=2, p=1.),
    TimeStretch(min_rate=0.85, max_rate=1.15, leave_length_unchanged=False, p=1.0)
    
])


In [ ]:
augmented_audio = augmentations(audio, sr)
augment
sd.play(augmented_audio, sr)